In [71]:
import scala.util.matching
import org.apache.spark.rdd.RDD

println("all good")

all good


data is in the format:

{
"conversionSymbol": "",
"volumeto": 6685400.72, 
"high": 9165.89, 
"low": 9144.6, 
"time": 1595030400, 
"volumefrom": 730.39, 
"close": 9151.68, 
"open": 9156.79, 
"conversionType": "direct"
}


In [120]:
case class TradingInfo (conversionSymbol: String, 
                        volumeto: Double, 
                        high: Double, 
                        low: Double, 
                        time: Double,
                        volumefrom: Double,
                        close: Double,
                        open: Double,
                        conversionType: String
                       )

val filename = "../Datasets/day_json.txt"
val file = sc.textFile(filename).flatMap(l => l.split("},")).
            map(x => x.replace("\\", "")).
            map(x => x.replace("[", "")).
            map(x => x.replace("{", "")).
            map(x => x.replace("\"", ""))

// THE CODE IS STUPID BUT IT WORKS 
// CONVERSION SYMBOL IS EMPTY BECAUSE I REMOVED \ FROM IT

file.take(5).foreach(println)

println("All good")

time: 1595037600, high: 9146.79, low: 9125.95, open: 9141.1, volumefrom: 612.19, volumeto: 5592905.32, close: 9144.83, conversionType: direct, conversionSymbol: 
 time: 1595041200, high: 9144.84, low: 9125.92, open: 9144.83, volumefrom: 536.94, volumeto: 4905635.56, close: 9139.76, conversionType: direct, conversionSymbol: 
 time: 1595044800, high: 9144.98, low: 9129.93, open: 9139.76, volumefrom: 501.97, volumeto: 4585206.62, close: 9143.94, conversionType: direct, conversionSymbol: 
 time: 1595048400, high: 9145.4, low: 9130.72, open: 9143.94, volumefrom: 342.17, volumeto: 3125247.16, close: 9137.95, conversionType: direct, conversionSymbol: 
 time: 1595052000, high: 9142.14, low: 9132.61, open: 9137.95, volumefrom: 295.63, volumeto: 2700215.73, close: 9139.16, conversionType: direct, conversionSymbol: 
All good


defined class TradingInfo
filename = ../Datasets/day_json.txt
file = MapPartitionsRDD[292] at map at <console>:34


MapPartitionsRDD[292] at map at <console>:34

In [107]:
val rdd = sc.textFile("../Datasets/odyssey.mb.txt")


//rdd.take(3).foreach(println)

rdd.flatMap(l => l.split(" } ")).
  map(x => x.replace("\\", "")).
  take(3).                
  foreach(println)

println("\n . \n ")

rdd.flatMap(l => l.split(" } ")).
  map(x => x).//replace("\\", "")).
  take(3).                
  foreach(println)


println("\n All good \n ")

Tell me, O muse, of that ingenious hero who travelled far and wide
after he had sacked the famous town of Troy. Many cities did he visit,/ and many were the nations with whose manners and customs he was acquainted;  moreover he suffered much by sea while trying to save his own life
and bring his men safely home; but do what he might he could not save

 . 
 
Tell me, O muse, of that ingenious hero who travelled far and wide
after he had sacked the famous town of Troy. Many cities did he visit,/ and many were the nations with whose manners and customs he was acquainted;\  moreover he suffered much by sea while trying to save his own life
and bring his men safely home; but do what he might he could not save

 All good 
 


rdd = ../Datasets/odyssey.mb.txt MapPartitionsRDD[244] at textFile at <console>:41


../Datasets/odyssey.mb.txt MapPartitionsRDD[244] at textFile at <console>:41

Compile Error: <console>:17: error: not found: type SparkContext
       val sc = new SparkContext(new SparkConf().setMaster("local[*]").setAppName("Example"))
                    ^
<console>:17: error: not found: type SparkConf
       val sc = new SparkContext(new SparkConf().setMaster("local[*]").setAppName("Example"))
                                     ^
